In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics
import pickle
from sklearn.preprocessing import  LabelEncoder

D:\Anaconda\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
def evauation_model(pred, y_val):
  return metrics.accuracy_score(pred, y_val)

In [3]:
def models_score(train_data, y_train, val_data, y_val):
    reg = SVC()
    reg.fit(train_data, y_train)
    pred = reg.predict(val_data)
    return metrics.accuracy_score(pred, y_val)

In [4]:
data = pd.read_csv(r"D:\Python\ArtyvisTechnologies\heart.csv")
categorical_features = ['ChestPainType', 'ST_Slope']
numeric_features = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']
eda_df = data.loc[:, numeric_features]
eda_df.columns

Index(['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak'], dtype='object')

In [5]:
def one_hot_encode(df, column_dict):
  for column, prefix in column_dict.items():
    dummies = pd.get_dummies(df[column], prefix = prefix)
    df = pd.concat([df, dummies], axis = 1)
    df = df.drop(column, axis = 1)
  return df

In [6]:
# data = one_hot_encode(data, dict(zip(categorical_features, ['CP', 'ST'])))

In [7]:
# data["Sex"] = data.groupby("Sex").ngroup()
# data["RestingECG"] = data.groupby("RestingECG").ngroup()
# data["ExerciseAngina"] = data.groupby("ExerciseAngina").ngroup()

In [8]:
label_encoder = LabelEncoder()

X = data.drop('HeartDisease', axis = 1)
y = data['HeartDisease']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [10]:
X_train['Sex'] = label_encoder.fit_transform(X_train['Sex'].values)
X_test['Sex'] = label_encoder.transform(X_test['Sex'].values)
np.save('classes_sex.npy', label_encoder.classes_)

X_train['ChestPainType'] = label_encoder.fit_transform(X_train['ChestPainType'].values)
X_test['ChestPainType'] = label_encoder.transform(X_test['ChestPainType'].values)
np.save('classes_chestpain.npy', label_encoder.classes_)

X_train['FastingBS'] = label_encoder.fit_transform(X_train['FastingBS'].values)
X_test['FastingBS'] = label_encoder.transform(X_test['FastingBS'].values)
np.save('classes_fastingbs.npy', label_encoder.classes_)

X_train['RestingECG'] = label_encoder.fit_transform(X_train['RestingECG'].values)
X_test['RestingECG'] = label_encoder.transform(X_test['RestingECG'].values)
np.save('classes_restingecg.npy', label_encoder.classes_)

X_train['ExerciseAngina'] = label_encoder.fit_transform(X_train['ExerciseAngina'].values)
X_test['ExerciseAngina'] = label_encoder.transform(X_test['ExerciseAngina'].values)
np.save('classes_exerciseangina.npy', label_encoder.classes_)

X_train['ST_Slope'] = label_encoder.fit_transform(X_train['ST_Slope'].values)
X_test['ST_Slope'] = label_encoder.transform(X_test['ST_Slope'].values)
np.save('classes_stslope.npy', label_encoder.classes_)

In [11]:
rs = pickle.load(open('rs.pkl', 'rb'))

In [12]:
X_train = pd.DataFrame(rs.fit_transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(rs.transform(X_test), columns = X_test.columns)

In [13]:
X_train

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,-0.384615,0.0,1.0,0.05,-0.830287,0.0,0.0,-0.363636,1.0,0.666667,0.0
1,-0.307692,0.0,0.0,1.00,0.224543,0.0,-1.0,-0.335664,0.0,1.400000,0.0
2,0.230769,0.0,1.0,1.00,-0.997389,1.0,0.0,0.923077,0.0,-0.200000,1.0
3,0.000000,0.0,1.5,-0.50,-0.527415,0.0,0.0,-0.083916,0.0,1.000000,1.0
4,-1.384615,0.0,1.0,1.00,-0.642298,0.0,0.0,0.895105,0.0,-0.333333,1.0
...,...,...,...,...,...,...,...,...,...,...,...
637,0.000000,-1.0,1.0,-1.10,0.475196,0.0,-1.0,0.755245,0.0,-0.333333,1.0
638,-0.153846,0.0,0.0,-0.50,-0.412533,0.0,0.0,0.279720,0.0,-0.333333,0.0
639,0.692308,0.0,0.0,0.50,-0.360313,0.0,-1.0,0.111888,1.0,2.333333,1.0
640,-1.153846,0.0,0.5,-0.50,-0.224543,0.0,0.0,0.559441,1.0,0.333333,0.0


In [14]:
svm_model = pickle.load(open(r"D:\Python\ArtyvisTechnologies\svcHeart.sav", 'rb'))
pred = svm_model.predict(X_test)
score = metrics.accuracy_score(pred, y_test) 

In [15]:
score

0.8659420289855072

In [16]:
# pred_trial = svm_model.predict(rs.transform([[20,1,120,175,0,1,120,0,0,0,1,0,0,0,0,1]]))
# pred_trial

In [17]:
# p = rs.([[20,1,120,175,0,1,120,0,0,0,1,0,0,0,0,1]])

In [18]:
# pred

In [19]:
# X_test.head(6)